In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import string

In [ ]:
file = 'DadosFinais.xlsx'

In [ ]:
res = pd.read_excel(file, 'Reservas')
prop = pd.read_excel(file, 'Propriedades')
guests = pd.read_excel(file, 'Hóspedes')
data = pd.read_excel(file, 'Reviews - geral')
reviews2 = pd.read_excel(file, 'Reviews - detalhe')

In [ ]:
data

,ReservationId,RatingValue,Comments,Language,Translated,Polarity,NetPromoterScore
0,1.879141e+09,8.0,"Very nice place, it was spacious and good loca...",__label__eng_Latn,nice place spacious welllocated communication ...,Positive,Good
1,1.879141e+09,8.0,"Casa muito espaçosa, com ótima vista e relativ...",__label__por_Latn,spacious house great view relatively close cen...,Positive,Good
2,1.879139e+09,10.0,We only stayed for a few days but the place is...,__label__eng_Latn,stayed day place pretty location great nice ar...,Positive,Very Good
3,1.879130e+09,8.0,"Un sitio agradable, lo mejor la terraza y las ...",__label__spa_Latn,pleasant site best part terrace view would per...,Positive,Good
4,1.879139e+09,10.0,"Wonderful spacious apt, comfortable furniture,...",__label__eng_Latn,wonderful spacious apt comfortable furniture s...,Positive,Very Good
...,...,...,...,...,...,...,...
11239,1.879192e+09,3.0,Infelizmente o apartamento não estava muito li...,__label__por_Latn,unfortunately apartment clean downstairs bathr...,Neutral,Neutral
11240,1.879179e+09,5.0,"Apartamento claro e limpo, muito boas instalaç...",__label__por_Latn,clear clean apartment good facility wonderful ...,Positive,Very Good
11241,1.879188e+09,2.0,o apartamento não vale o dinheiro.esta é minha...,__label__por_Latn,apartment worth money fourth time lisbon cheap...,Negative,Bad
11242,1.879179e+09,5.0,"A comunicação foi muito suave, o lugar é incrí...",__label__por_Latn,communication smooth place incredible staff fr...,Positive,Very Good


In [ ]:
data.dropna(inplace= True)

In [ ]:
data.isna().sum()

ReservationId       0
RatingValue         0
Comments            0
Language            0
Translated          0
Polarity            0
NetPromoterScore    0
dtype: int64

## Aspect extraction

In [ ]:
import os
from google.colab import userdata
GROQ_API_TOKEN = userdata.get('GROQ_API_KEY')

os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from groq import Groq

# Initialize Groq client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Function to translate using Groq
def llama3_70b(prompt, temperature=0.0):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192",
        temperature=temperature,
    )
    return chat_completion.choices[0].message.content

# Translate function for each row
def aspect_extraction(row):
    comment = row['Translated']
    prompt = "Recognize all aspect terms with their corresponding sentiment polarity in the given review. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Answer in the format [\"aspect\", \"sentiment\"] without any explanation. If no aspect term exists, then only answer \"[]\"."
    prompt_text = f"{prompt} '''{comment}'''"
    translated_comment = llama3_70b(prompt_text)
    return translated_comment

''






''

In [ ]:
data1 = data[:2000]
data2 = data[2000:4000]
data3 = data[4000:6000]
data4 = data[6000:8000]
data5 = data[8000:10000]
data6 = data[10000:]

In [ ]:
# Apply translate_row function to each row and store the result in 'EngTrad' column
tqdm.pandas(desc="Aspect Extraction")
data1['Aspects'] = data1.progress_apply(aspect_extraction, axis=1)

Aspect Extraction: 100%|██████████| 2000/2000 [1:19:32<00:00,  2.39s/it]
<ipython-input-11-1d6390d13ed1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Aspects'] = data1.progress_apply(aspect_extraction, axis=1)


In [ ]:
data1.to_excel('ae1.xlsx',index=False)

In [ ]:
tqdm.pandas(desc="Aspect Extraction")
data2['Aspects'] = data2.progress_apply(aspect_extraction, axis=1)

Aspect Extraction: 100%|██████████| 2000/2000 [1:20:37<00:00,  2.42s/it]
<ipython-input-13-a01373154759>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Aspects'] = data2.progress_apply(aspect_extraction, axis=1)


In [ ]:
data2.to_excel('ae2.xlsx',index=False)

In [ ]:
tqdm.pandas(desc="Aspect Extraction")
data3['Aspects'] = data3.progress_apply(aspect_extraction, axis=1)

Aspect Extraction: 100%|██████████| 2000/2000 [1:21:14<00:00,  2.44s/it]
<ipython-input-15-26427d1ae91d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['Aspects'] = data3.progress_apply(aspect_extraction, axis=1)


In [ ]:
data3.to_excel('ae3.xlsx',index=False)

In [ ]:
tqdm.pandas(desc="Aspect Extraction")
data4['Aspects'] = data4.progress_apply(aspect_extraction, axis=1)

Aspect Extraction: 100%|██████████| 2000/2000 [3:30:51<00:00,  6.33s/it]
<ipython-input-17-3d1bafbc7ed6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['Aspects'] = data4.progress_apply(aspect_extraction, axis=1)


In [ ]:
data4.to_excel('ae4.xlsx',index=False)

In [ ]:
tqdm.pandas(desc="Aspect Extraction")
data5['Aspects'] = data5.progress_apply(aspect_extraction, axis=1)

Aspect Extraction: 100%|██████████| 2000/2000 [3:37:07<00:00,  6.51s/it]
<ipython-input-19-174d0cf48a74>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data5['Aspects'] = data5.progress_apply(aspect_extraction, axis=1)


In [ ]:
data5.to_excel('ae5.xlsx',index=False)

In [ ]:
tqdm.pandas(desc="Aspect Extraction")
data6['Aspects'] = data6.progress_apply(aspect_extraction, axis=1)

Aspect Extraction:  83%|████████▎ | 461/558 [49:39<10:23,  6.42s/it]

In [ ]:
data6.to_excel('ae6.xlsx',index=False)

In [ ]:
#data['Aspects'] = data['Aspects'].str.replace('Here is the list of aspect terms with their corresponding sentiment polarity:\n\n', '')

In [ ]:
file = 'ae1.xlsx'

In [ ]:
data1 = pd.read_excel(file)

In [ ]:
file = 'ae2.xlsx'

In [ ]:
data2 = pd.read_excel(file)

In [ ]:
file = 'ae3.xlsx'

In [ ]:
data3 = pd.read_excel(file)

In [ ]:
file = 'ae4.xlsx'

In [ ]:
data4 = pd.read_excel(file)

In [ ]:
file = 'ae5.xlsx'

In [ ]:
data5 = pd.read_excel(file)

In [ ]:
file = 'ae6.xlsx'

In [ ]:
data6 = pd.read_excel(file)

In [ ]:
dfs = [data1, data2, data3, data4, data5, data6]

# Concatenate (merge) all DataFrames in the list together
merged_df = pd.concat(dfs, ignore_index=False)

In [ ]:
data = merged_df

In [ ]:
data

,ReservationId,RatingValue,Comments,Language,Translated,Polarity,NetPromoterScore,Aspects
0,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"[""place"", ""positive""], [""location"", ""positive""..."
1,1879140870,8,"Casa muito espaçosa, com ótima vista e relativ...",__label__pt,spacious house great view relatively close cen...,Positive,Good,"[""house"", ""positive""], [""view"", ""positive""], [..."
2,1879139425,10,We only stayed for a few days but the place is...,__label__en,stayed day place pretty location great nice ar...,Positive,Very Good,Here is the list of aspect terms with their co...
3,1879129551,8,"Un sitio agradable, lo mejor la terraza y las ...",__label__es,pleasant site best part terrace view would per...,Positive,Good,"[[""site"", ""positive""], [""terrace view"", ""posit..."
4,1879139286,10,"Wonderful spacious apt, comfortable furniture,...",__label__en,wonderful spacious apt comfortable furniture s...,Positive,Very Good,"[""apt"", ""positive""], [""furniture"", ""positive""]..."
...,...,...,...,...,...,...,...,...
554,1879192185,3,Infelizmente o apartamento não estava muito li...,__label__pt,unfortunately apartment clean downstairs bathr...,Neutral,Neutral,"[[""apartment"", ""positive""], [""bathroom"", ""nega..."
555,1879178889,5,"Apartamento claro e limpo, muito boas instalaç...",__label__pt,clear clean apartment good facility wonderful ...,Positive,Very Good,"[[""apartment"", ""positive""], [""facility"", ""posi..."
556,1879187989,2,o apartamento não vale o dinheiro.esta é minha...,__label__pt,apartment worth money fourth time lisbon cheap...,Negative,Bad,Here is the list of aspect terms with their co...
557,1879178958,5,"A comunicação foi muito suave, o lugar é incrí...",__label__pt,communication smooth place incredible staff fr...,Positive,Very Good,"[[""communication"", ""positive""], [""place"", ""pos..."


## Cleaning

In [ ]:
data['Aspects'] = data['Aspects'].str.replace('Here is the list of aspect terms with their corresponding sentiment polarity:\n\n', '')
data['Aspects'] = data['Aspects'].str.replace('Here is the answer:\n\n', '')
data['Aspects'] = data['Aspects'].str.replace('\n', '')
data['Aspects'] = data['Aspects'].str.replace('\n\n', '')
data['Aspects'] = data['Aspects'].str.replace('[', '').str.replace(']', '')


In [ ]:
data = data[data['Aspects'] != '']
data = data.reset_index(drop=True)

In [ ]:
data.tail(50)

,ReservationId,RatingValue,Comments,Language,Translated,Polarity,NetPromoterScore,Aspects
10443,1879181564,4,O apartamento está idealmente localizado próxi...,__label__pt,apartment ideally located near place du commer...,Positive,Good,"""apartment"", ""positive"", ""location"", ""positive..."
10444,1879182400,5,Dificilmente há um apartamento com uma vista m...,__label__pt,hard find apartment better view city although ...,Positive,Very Good,"""apartment"", ""positive"", ""view"", ""positive"", ""..."
10445,1879182655,4,"A comunicação com o anfitrião é muito boa, sem...",__label__pt,communication host good always available airpo...,Positive,Good,"""communication"", ""positive""""host"", ""positive""""..."
10446,1879184914,4,"Localização muito agradável, ideal para explor...",__label__pt,pleasant location ideal exploring city studio ...,Positive,Good,"""location"", ""positive"", ""studio"", ""neutral"", ""..."
10447,1879188428,5,Ótima localização no coração de Alfama. O loca...,__label__pt,great location heart alfama place look like po...,Positive,Very Good,"""location"", ""positive"", ""place"", ""positive"", ""..."
10448,1879188612,2,O anúncio não está de acordo com a realidade d...,__label__pt,announcement match reality product plan many p...,Negative,Bad,"""announcement"", ""negative"", ""product"", ""negati..."
10449,1879189999,5,"Grande apartamento! Muito central, limpo e esp...",__label__pt,great apartment central clean spacious really ...,Positive,Very Good,"""apartment"", ""positive"", ""location"", ""positive..."
10450,1879190591,4,"A localização é brilhante, curta caminhada até...",__label__pt,location brilliant short walk center ericeira ...,Positive,Good,"""location"", ""positive"", ""walk"", ""positive"", ""c..."
10451,1879190965,1,Eu não recomendaria esta Airbnb. É a minha pio...,__label__pt,would recommend airbnb worst experience far fa...,Negative,Awfull,"""apartment"", ""negative"", ""company"", ""negative""..."
10452,1879191237,2,"excelente localização, acomodações limpas. no ...",__label__pt,excellent location clean accommodation however...,Negative,Bad,"""location"", ""positive"", ""accommodation"", ""posi..."


In [ ]:
data['Aspects_Polarity'] = data['Aspects']

In [ ]:
data['Aspects'] = data['Aspects'].str.replace('"positive"', '')
data['Aspects'] = data['Aspects'].str.replace('"neutral"', '')
data['Aspects'] = data['Aspects'].str.replace('"negative"', '')

In [ ]:
pattern = r'\b(?!positive|neutral|negative)\w+\b'

# Replace words in 'Aspects_Polarity' column that don't match the pattern with an empty string
data['Aspects_Polarity'] = data['Aspects_Polarity'].apply(lambda x: re.sub(pattern, '', x))

# Reset the index if needed
data.reset_index(drop=True, inplace=True)

In [ ]:
data['Aspects'] = data['Aspects'].str.replace(',', '')
data['Aspects'] = data['Aspects'].apply(lambda x: ', '.join(x.split()))

In [ ]:
def remove_punctuation(text):
    return re.sub(f'[{string.punctuation}]', '', text)


data['Aspects_Polarity'] = data['Aspects_Polarity'].apply(remove_punctuation)

In [ ]:
def split_into_quoted_words(text):
    words = text.split()
    return ' '.join(f'"{word}"' for word in words)

In [ ]:
data['Aspects_Polarity'] = data['Aspects_Polarity'].apply(split_into_quoted_words)
data['Aspects_Polarity'] = data['Aspects_Polarity'].apply(lambda x: ', '.join(x.split()))

In [ ]:
def process_aspects(aspects):
    # Regular expression to find phrases within double quotation marks
    pattern = r'"([^"]+)"'

    # Replace commas inside the matched phrases
    def replace_commas(match):
        return match.group(0).replace(',', '')

    # Apply the regular expression substitution to each aspect
    processed_aspects = re.sub(pattern, replace_commas, aspects)

    return processed_aspects

# Assuming data is your DataFrame and 'Aspects' is the column containing the aspects
data['Aspects'] = data['Aspects'].apply(process_aspects)

In [ ]:
data.head(10)

,ReservationId,RatingValue,Comments,Language,Translated,Polarity,NetPromoterScore,Aspects,Aspects_Polarity
0,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""place"", ""location"", ""communication"", ""home"", ...","""positive"", ""positive"", ""positive"", ""positive""..."
1,1879140870,8,"Casa muito espaçosa, com ótima vista e relativ...",__label__pt,spacious house great view relatively close cen...,Positive,Good,"""house"", ""view"", ""area"", ""host"", ""house"", ""bla...","""positive"", ""positive"", ""positive"", ""positive""..."
2,1879139425,10,We only stayed for a few days but the place is...,__label__en,stayed day place pretty location great nice ar...,Positive,Very Good,"""location"", ""shop"", ""cafe"", ""open space"", ""wel...","""positive"", ""positive"", ""positive"", ""positive""..."
3,1879129551,8,"Un sitio agradable, lo mejor la terraza y las ...",__label__es,pleasant site best part terrace view would per...,Positive,Good,"""site"", ""terrace view"", ""faucet"", ""blind"", ""li...","""positive"", ""positive"", ""negative"", ""negative""..."
4,1879139286,10,"Wonderful spacious apt, comfortable furniture,...",__label__en,wonderful spacious apt comfortable furniture s...,Positive,Very Good,"""apt"", ""furniture"", ""location"", ""outdoor terra...","""positive"", ""positive"", ""positive"", ""positive""..."
5,1879139607,8,"Good and decent apartment, clean, but widows a...",__label__en,good decent apartment clean window transparent...,Positive,Good,"""apartment"", ""window"", ""living area""","""positive"", ""positive"", ""positive"""
6,1879130683,8,"Le logement est très très bien situé, propre d...",__label__fr,accommodation well located clean overall also ...,Positive,Good,"""accommodation"", ""location"", ""concierge servic...","""positive"", ""positive"", ""positive"", ""positive""..."
7,1879138192,10,"Le studio était très propre, tout est encore f...",__label__fr,studio clean everything still brand new except...,Positive,Very Good,"""studio"", ""location"", ""volume"", ""apartment"", ""...","""positive"", ""positive"", ""positive"", ""positive""..."
8,1879137569,8,Apparemment idéalement placé clair et spacieux...,__label__fr,apparently ideally located bright spacious wel...,Positive,Good,"""location"", ""accommodation"", ""space"", ""insulat...","""positive"", ""positive"", ""positive"", ""positive""..."
9,1879118676,8,"Ótima estadia! Perto do centro, limpo, a comun...",__label__pt,great stay close center clean communication ho...,Positive,Good,"""stay"", ""center"", ""communication"", ""host""","""positive"", ""positive"", ""positive"", ""positive"""


In [ ]:
data['Aspects'] = data['Aspects'].str.split(",")
data['Aspects_Polarity'] = data['Aspects_Polarity'].str.split(",")

In [ ]:
lengths_aspects = data['Aspects'].apply(len)
lengths_polarity = data['Aspects_Polarity'].apply(len)

# Check for rows where lengths don't match
mismatched_rows = data[lengths_aspects != lengths_polarity]
len(mismatched_rows)

172

In [ ]:
data = data[lengths_aspects == lengths_polarity]

# Explode the DataFrame
df = data.explode(['Aspects', 'Aspects_Polarity'], True)

In [ ]:
df.head(8)

,ReservationId,RatingValue,Comments,Language,Translated,Polarity,NetPromoterScore,Aspects,Aspects_Polarity
0,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""place""","""positive"""
1,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""location""","""positive"""
2,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""communication""","""positive"""
3,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""home""","""positive"""
4,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""reply""","""positive"""
5,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""issue""","""negative"""
6,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""cockroach""","""negative"""
7,1879140693,8,"Very nice place, it was spacious and good loca...",__label__en,nice place spacious welllocated communication ...,Positive,Good,"""ant""","""negative"""


In [ ]:
punctuations = ['"']  # Including single quote

# Remove punctuations from the 'Aspect' column
for punctuation in punctuations:
    df['Aspects'] = df['Aspects'].str.replace(punctuation, '')
    df['Aspects_Polarity'] = df['Aspects_Polarity'].str.replace(punctuation, '')

In [ ]:
df['Aspects'].value_counts()

Aspects
 location               4480
apartment               3876
 apartment              2359
location                1860
 bathroom                803
                        ... 
 car racing                1
 city coast                1
store                      1
 train/metro station       1
 drainage smell            1
Name: count, Length: 6900, dtype: int64

In [ ]:
df['Aspects'] = df['Aspects'].str.lstrip()

In [ ]:
df['Aspects'].value_counts()

Aspects
location                 6340
apartment                6235
place                    1298
view                      882
bathroom                  837
                         ... 
local host experience       1
armaçao de pêra beach       1
residential complex         1
hospitality team            1
drainage smell              1
Name: count, Length: 6377, dtype: int64

In [ ]:
with pd.ExcelWriter('DadosFinais2.xlsx') as writer:
    res.to_excel(writer, sheet_name="Reservas", index=False)
    prop.to_excel(writer, sheet_name="Propriedades", index=False)
    guests.to_excel(writer, sheet_name="Hóspedes", index=False)
    df.to_excel(writer, sheet_name="Reviews - geral", index=False)
    reviews2.to_excel(writer, sheet_name="Reviews - detalhe", index=False)

In [ ]:
len(df)

63747